## Import the data

Use Pandas to import the data and prepare it for the models

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('Dataset/dataset_phishing.csv')

#print null
print(df.isnull().sum())

#remove all null
df = df.dropna()

#describe statistical data. To stdout
with pd.option_context('display.max_columns', 40):
    print(df.describe(include='all'))

#describe statistical data to text file: out.txt
with open('out.txt', 'w') as f:
    with pd.option_context('display.max_columns', 40):
        print(df.describe(include='all'),file=f)
        
#shape of data
print('Number of rows are',df.shape[0], 'and number of columns are ',df.shape[1])

#look at data types of columns
print(df.info())

#Pair plot
plt.figure(figsize=(20,20))
sns.pairplot(df)
plt.savefig('./Figures/pairplot.png')
plt.show()